In [ ]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!pip install mecab
!pip install mecab-python3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!ln -s /etc/mecabrc /usr/local/etc/mecabrc


     |████████████████████████████████| 71kB 7.5MB/s 
  Created wheel for mecab: filename=mecab-0.996.3-cp37-cp37m-linux_x86_64.whl size=141787 sha256=81c8739ca15d1238b2da64922d611247913190c5bf8d514581462e19ccc073b1
  Stored in directory: /root/.cache/pip/wheels/8f/95/18/2c71c144b3c41ba34f532aa8b6531d1d93980a71db8b3d200b
Successfully built mecab
     |████████████████████████████████| 491kB 32.6MB/s 
Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
import csv
import json
import MeCab
import pandas as pd

In [ ]:
def conv_neologd(input_file, output_file):
	# ファイルをオープン
	with open(input_file, "r", encoding='utf-8') as f:
		train = json.load(f)

	# Mecabの定義
	mtagger = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

	# JSONの要素ごとに分解
	# 要素ごとに取得、分かち書き、更新を行う
	for data in train["data"]:
		paragraphs = data["paragraphs"]
		for paragraph in paragraphs:
			# 本文(context)
			context = paragraph["context"]
			context = mtagger.parse(context).replace(' \n', '')
			paragraph["context"] = context
			for qas in paragraph["qas"]:
				# 質問(question)
				question = qas["question"]
				question = mtagger.parse(question).replace(' \n', '')
				qas["question"] = question

				# 回答(text)
				text = qas["answers"][0]["text"]
				text = mtagger.parse(text).replace(' \n', '')
				qas["answers"][0]["text"] = text

				# 回答位置(answar_start)
				# 回答(text)が本文(context)内のどこにあるのかを探索して更新
				qas["answers"][0]["answer_start"] = context.find(text)

	# ファイル出力
	with open(output_file, 'w', encoding='utf-8') as f:
	  json.dump(train, f, ensure_ascii=False)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
input_file1 = "/content/gdrive/MyDrive/Colab Notebooks/drive_data/DDQA-1.0_RC-QA_dev.json"
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/drive_data/DDQA-1.0_RC-QA_dev_neologd.json"
input_file2 = "/content/gdrive/MyDrive/Colab Notebooks/drive_data/DDQA-1.0_RC-QA_train.json"
output_file2 = "/content/gdrive/MyDrive/Colab Notebooks/drive_data/DDQA-1.0_RC-QA_train_neologd.json"

In [ ]:
conv_neologd(input_file1, output_file1)
conv_neologd(input_file2, output_file2)